# Examination dynamics

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../../src/SEIR/')
sys.path.insert(1, '../../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows': 
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIRQ import SEIR
from Quarantine import Quarantine
from Quarantine import Exams

Linux


# Beta, delay and exam rate
exam rate i  examrate = np.arange(1,21,1)*population/100# Persons per day  
beta j  beta = np.arange(0.11,0.21,0.01)  
delay days k  T_T = [1,2,3,4,5]  


In [2]:
mu = 0 # E0/I0 initial rate
sigma = 0.2
gamma = 0.1

SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 100000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Quarantine
s1 = Quarantine(1)

In [3]:
duty = 1
period = 1
testaccuracy=0.9
lambda_Tr = 0 # traceability
lambda_Q=1 # Proportion of effectively quarantined detected infected
T_Q = 14 # Time in quarantine

# Exam Rate
examrate = np.arange(1,21,1)*population/100# Persons per day

# Delay Time
T_T = [1,2,3,4,5]
# Beta
beta = np.arange(0.11,0.21,0.01)

psi = [Exams(i,period,duty) for i in examrate] 

In [4]:
# Define Simulations
sims_reference = []
for i in range(len(beta)):
    sims_reference.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta[i],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = 1,T_Q = T_Q,lambda_Tr=lambda_Tr))

In [17]:
levels = np.arange(0,105,5)

## Reference Simulation
Here we simulate situations without exams that will serve as references

In [5]:
# For parallel simulation
def simulate_reference(simulation,tsim):
    simulation.integr_sci(0,tsim,0.1)
    return simulation

# Reference simulation
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation_reference = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate_reference)(sims_reference[i],tsim) for i in range(len(beta)))

print('simulation finished')


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.1s
[Parallel(n_jobs=12)]: Done   2 out of  10 | elapsed:    1.2s remaining:    4.6s
[Parallel(n_jobs=12)]: Done   3 out of  10 | elapsed:    1.3s remaining:    3.0s
[Parallel(n_jobs=12)]: Done   4 out of  10 | elapsed:    1.3s remaining:    2.0s
[Parallel(n_jobs=12)]: Done   5 out of  10 | elapsed:    1.3s remaining:    1.3s
[Parallel(n_jobs=12)]: Done   6 out of  10 | elapsed:    1.4s remaining:    0.9s
[Parallel(n_jobs=12)]: Done   7 out of  10 | elapsed:    1.4s remaining:    0.6s
[Parallel(n_jobs=12)]: Done   8 out of  10 | elapsed:    1.4s remaining:    0.4s
[Parallel(n_jobs=12)]: Done  10 out of  10 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=12)]: Done  10 out of  10 | elapsed:    1.5s finished
simulation finished


In [6]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta[j],sigma=sigma,gamma=gamma,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T[k],T_Q = T_Q,lambda_Tr=lambda_Tr))
        aux.append(aux2)
    sims.append(aux)

In [7]:
#%%capture
# To show the cell's output comment the former line
# For parallel simulation
def simulate(simulation,k,tsim):
    simulation[k].integr_sci(0,tsim,0.1)
    return simulation[k]

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i][j],k,tsim) for k in range(len(sims[i][j]))))
    simulation.append(aux)
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0433s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0408s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0576s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0623s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0404s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0401s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0447s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0440s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0635s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0565s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

...

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0421s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0409s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0483s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0435s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0518s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0472s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0440s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0424s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0463s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0486s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Para

In [8]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(100*simulation[i][j][k].peak/population)
        aux.append(aux)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):
            aux2.append(simulation[i][j][k].peak_t)
        aux.append(aux2)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(simulation[i][j][k].prevalence_total[-1])  
        aux.append(aux2)
    prevalence.append(aux)
prevalence = np.array(prevalence)
    
# Calculate indicators
peakvsnoexam = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(100*simulation[i][j][k].peak/simulation_reference[j].peak)
        aux.append(aux2)
    peakvsnoexam.append(aux)
peakvsnoexam = np.array(peakvsnoexam)
# Calculate indicators
totinfected = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(beta)):
        aux2 = []
        for k in range(len(T_T)):        
            aux2.append(100*simulation[i][j][k].I_ac[-1]/simulation_reference[j].I_ac[-1])
        aux.append(aux2)
    totinfected.append(aux)

totinfected = np.array(totinfected)

### QA: Population consistency

In [14]:
i = 0
j = 0
k = 0

In [15]:
max(simulation[i][j][k].S+simulation[i][j][k].E+simulation[i][j][k].I+simulation[i][j][k].R-population+simulation[i][j][k].Q+simulation[i][j][k].I_T)

5.69868197075607e-09

# Plots

## Plot 1: Grid Epidemic plots


In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(beta)))
n = 4
j = 4
fig, axs = plt.subplots(int(len(examrate)/n)+(int(len(examrate))%n > 0)*1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference[j].t,simulation_reference[j].I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for k in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j][k].t,simulation[i][j][k].I_d,label='Delay days: '+str(T_T[k]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(round(examrate[i]/population*100))+'%')
fig.suptitle('New Daily Infected')

## Plot 2: Tests vs Delay para distintos beta

In [ ]:
# Contour Plot
n = 3
fig,ax=plt.subplots(4,3)
for j in range(len(beta)):    
    cp = ax[int(j/n),j%n].contourf(T_T,examrate*100/population,totinfected[:,j,:],levels)     
    ax[int(j/n),j%n].set_title('Beta: '+str(round(beta[j],2)))
    ax[int(j/n),j%n].set_xlabel('Delay days')
    ax[int(j/n),j%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(j/n),j%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

In [ ]:
# Contour Plot
n = 3
fig,ax=plt.subplots(4,3)
for j in range(len(beta)):    
    cp = ax[int(j/n),j%n].contourf(T_T,examrate*100/population,100*np.array(prevalence)[:,j,:],np.arange(0,105,5))
    ax[int(j/n),j%n].set_title('Beta: '+str(round(beta[j],2)))
    ax[int(j/n),j%n].set_xlabel('Delay days')
    ax[int(j/n),j%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(j/n),j%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

In [ ]:
np.array)prevalence[]

## Plot 3: Test vs Beta para distintos delay

In [ ]:
# Contour Plot
n = 3
fig,ax=plt.subplots(2,3)
for k in range(len(T_T)):    
    cp = ax[int(k/n),k%n].contourf(beta,examrate*100/population,totinfected[:,:,k],levels)     
    ax[int(k/n),k%n].set_title('Exams delay: '+str(T_T[k]))
    ax[int(k/n),k%n].set_xlabel('beta')
    ax[int(k/n),k%n].set_ylabel('Normalized (%) Exams Rate')
    fig.colorbar(cp, ax=ax[int(k/n),k%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

## Plot4: Delay vs beta para distintas tasas de exámenes

In [ ]:
# Contour Plot
n = 5
fig,ax=plt.subplots(4,5)
for i in range(len(examrate)):    
    cp = ax[int(i/n),i%n].contourf(T_T,beta,totinfected[i,:,:],levels)     
    ax[int(i/n),i%n].set_title('Exam rate: '+str(100*(examrate[i])/population)+'%')
    ax[int(i/n),i%n].set_xlabel('Delay days')
    ax[int(i/n),i%n].set_ylabel('beta')    
    fig.colorbar(cp, ax=ax[int(i/n),i%n])
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
fig.suptitle('Total infected proportion to the dynamic with no exams')

## Plot Paper

In [ ]:
totinfected[-1]

In [60]:
levels = [0,5,100]
colors = plt.cm.jet(np.linspace(0,1,len(beta)))

In [69]:
# Contour Plot
n = 3
fig,ax=plt.subplots(1,1)
#ax.set_title('Beta: '+str(round(beta[j],2)))
ax.set_xlabel('Delay days')
ax.set_ylabel('Normalized (%) Exams Rate')  

for j in range(len(beta)):
    cp = ax.contour(T_T,examrate*100/population,totinfected[:,j,:],levels,colors=colors[j])
    ax.clabel(cp, inline=1, fontsize=10,fmt=str(round(beta[j],2)))
#fig.legend(loc=0)
fig.suptitle('Total infected proportion compared to the dynamic with no exams')
#fig.colorbar(cp)
plt.show() 
#fig.colorbar(cp) # Add a colorbar to a plot
#fig.colorbar(cp, ax=ax)
